In [3]:
#imports
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

# Constants (All in SI units) and Data

Relevant physical constants

In [ ]:
class constants:
    """
    Parameters:
        k_B: Boltzmann constant in J/K
        hbar: reduced Planck's constant
        h: Planck constant
        amu: 1 amu in kg
        c: speed of light in m/s
        mu_B: Bohr magneton in J/T
        a_0: Bohr radius in m
        q_e: electron charge
        mu_0: vacuum permeability
        epsilon_0: vacuum permittivity
    """
    k_B = 1.380649e-23  # Boltzmann constant in J/K
    hbar = 1.054572e-34  # reduced Planck's constant
    h = hbar * 2 * np.pi  # Planck constant
    amu = 1.660539066e-27  # 1 amu in kg
    c = 299792458  # speed of light in m/s
    mu_B = 9.2740100657e-24  # Bohr magneton in J/T
    a_0 = 5.29177210544e-11  # Bohr radius in m
    q_e = 1.602176634-19  # electron charge
    mu_0 = 1.25663706127e-6  # vacuum permeability
    epsilon_0 = 8.8541878188e-12  # vacuum permittivity

imported data as numpy arrays

In [26]:
CsVsBPath = "/Users/mouhamedmbengue/Desktop/code/chinLabCode/LiCs-CodebaseV2/data/Cs_a_vs_B.txt"
LiCs_aVsBPath = "/Users/mouhamedmbengue/Desktop/code/chinLabCode/LiCs-CodebaseV2/data/LiCs_a_vs_B.txt"

CsVsBData2 = np.loadtxt(fname=CsVsBPath, delimiter=",", skiprows=1)
LiCs_aVsBData = np.loadtxt(fname=LiCs_aVsBPath, delimiter=",", skiprows=1)

# Atomic Properties

Class for storing properties of Cesium 133

In [64]:
class Cs133:
    """This class stores atomic properties of cesium-133."""
    mass = 132.905451931 * constants.amu  # atomic mass in kg
    I = 7/2  # nuclear spin
    g_I = -0.00039885395  # nuclear g-factor

    class GS: #ground state
        g_j = 2.002540261 #fine structure landé g-factor
        aHF = constants.h * 2.2981579425e9 #hyperfine structure constant in joules
        #2298.1579425 #hyperfine structure constant in MHz

    class D1: #D1 transition
        omega = 2 * np.pi * 335.116048807e12  # angular frequency in Hz
        Gamma = 2 * np.pi * 4.575e6  # decay rate in Hz
        wavelength = 894.59295987e-9  # wavelength (in vaccum) in m
        g_j = 0.665900 #fine structure landé g-factor
        aHF = 291.9201 #hyperfine structure constant in MHz

    class D2: #D2 transition
        omega = 2 * np.pi * 351.72571850e12  # angular frequency in Hz
        Gamma = 2 * np.pi * 5.234e6  # decay rate in Hz
        wavelength = 852.34727582e-9  # wavelength (in vacuum) in m
        g_j = 1.33408749 #fine structure landé g-factor
        aHF = 50.28827 #hyperfine structure constant in MHz

Class for storing properties of Lithium 6

In [ ]:
class Li6:
    """This class stores atomic properties of lithium-6."""
    mass = 6.0151214 * constants.amu  # atomic mass in kg
    I = 1  # nuclear spin
    g_I = -0.000447654  # nuclear g-factor

    class GS: #ground state 
        aHF = 152.1368407 # hyperfine structure constrant in MHz
        g_j = 2.0023010 #Total electronic g-factor

    class D1: #D1 transition
        omega = 2 * np.pi * 446.789634e12 # angular frequency in Hz
        Gamma = 2 * np.pi * 5.8724e6 # decay rate in Hz
        wavelength = 670.992421e-9 # wavelength in m
        g_j = 0.6668  #Total electronic g-factor
        aHF = 17.386 # hyperfine structure constant in MHz
    
    class D2: #D2 transition
        omega = 2 * np.pi * 446.799677e12  # angular frequency in Hz
        Gamma = 2 * np.pi * 5.8724e6 # decay rate in Hz
        wavelength = 670.9977338e-9  # wavelength (in vacuum) in m
        g_j = 1.335 #Total electronic g-factor
        aHF = -1.155 # hyperfine structure constant in MHz

# Calculations

## Breit-Rabi Energy

The Breit-rabi energy is computed using the following formula:

$$ E_{\ket{J=1/2, m_J, I, m_I}}  =  - \frac{\Delta E_{hfs}}{2(2I +1)} + g_i \mu_B m B \pm  \frac{\Delta E_{hfs}}{2} (1 + \frac{4mx}{2I + 1} + x^2)^{1/2}$$

taken from:

https://steck.us/alkalidata/cesiumnumbers.pdf

where:

$$ \Delta E_{hfs} = A_{hfs}(I + 1/2) $$

is the hyper fine splitting energy, and:

$$ x = \frac{(g_J - g_I)\mu_B B}{\Delta E_{hfs}} $$

For the purposes of this function:

$ coef1 = \frac{\Delta E_{hfs}}{2(2I +1)} $

$ coef2 = g_I \mu_B m B$

$coef3 = \frac{\Delta E_{hfs}}{2} (1 + \frac{4mx}{2I + 1} + x^2)^{1/2}$

In [94]:
def breitRabi(B, m, pm:str, atomType:int, freq:bool):
    """
    Function for computing the breit-rabi energy for Li6 or Cs133

    Parameters:
        B: Magnetic field in gauss
        m: Total magnetic quantum number
        pm: "p" or "m" decides between plus or minus in the formula
        atomType: atomic species, 6 for lithium or 133 for cesium
        freq: parameter to decide whether or not to return the answer as a frequency
    
    Returns:
        eBR: breit-rabi energy in joules or Hz depending on the freq parameter
    """

    #convert the field value to Tesla
    B *= 1e-4

    #define relevant constants depending on atom type
    if atomType == 6: #Lithium
        g_I = Li6.g_I
        g_J = Li6.GS.g_j
        a_HF = Li6.GS.aHF
        I = Li6.I
    
    elif atomType == 133: #Cesium
        g_I = Cs133.g_I
        g_J = Cs133.GS.g_j
        a_HF = Cs133.GS.aHF 
        I = Cs133.I
    
    else:
        return "Error: atomType must be 6 or 133"
    

    #determine whether we are in the plus state or minus state
    if pm == "p":
        sign = 1
    elif pm == "m":
        sign = -1
    else:
        return "Error: pm must be p or m"
    
    #compute the hyperfine splitting energy
    E_hfs = a_HF * (I + 1/2)

    #compute the x-factor
    x = ((g_J - g_I) * constants.mu_B * B)/E_hfs

    # compute the three coefficients
    coef1 = E_hfs/( 2*(2*I + 1) )

    coef2 = g_I * constants.mu_B * m * B

    coef3 = E_hfs/2 * ( 1 + (4*m*x)/(2*I +1) + x**2)**(1/2)

    #compute breit-rabi energy
    E_br =  -coef1 + coef2 + sign*coef3

    #determine whether to return the answer as a frequency (Hz) or energy (J)
    if freq:
        return E_br / constants.h
    else:
        return E_br


## Scattering length calculations

Function for returning the scattering length of two Cs atoms in the |3, 3> state.

The data is from:
    
    Berninger et. al. "Feshbach resonances, weakly bound molecular states, and coupled-channel potentials for cesium at high magnetic fields" Phys. Rev. A 87, 032517 (2013)

It is stored in Cs_a_vs_B.txt.

In [22]:
def aCsCs(B:float):
    """
    aCsCs - returns the scattering length of two Cs atoms in the |3, 3> state.

    Parameters:
        B: The magnetic field strength in Gauss.
    
    Returns:
        a: The scattering length in Bohr radii.
    
    """
    #get the magnetic field strength and scatering length
    BCsData = CsVsBData2[:, 0]
    Cs_a = CsVsBData2[:, 1]

    #interpolate the data using a cubic spline
    spline = sp.interpolate.CubicSpline(BCsData, Cs_a, extrapolate=False)

    #return the scattering length value for a given field strength
    return spline(B)

Function for calculating the scattering length of two Li atoms of various species for a given magnetic field. 

Possible species inputs are "ab", "ac", or "bc" which represent the |1, 2>, |1, 3>, and |2, 3> states respectively.

Values are from table II of:

    Precise Characterization of $^{6}\mathrm{Li}$ Feshbach Resonances Using Trap-Sideband-Resolved RF Spectroscopy of Weakly Bound Molecules" by Zurn, et. al. PRL 110 13 135301 (2013)


In [122]:
def aLiLi(B, species):
    """
    aLiLi - returns the Li Li scattering lengths at field B between the species given in species ('ab', 'ac', or 'bc'). 

    Parameters:
        B: The magnetic field strength in Gauss.

        species: The state of the lithium atoms 'ab', 'ac', or 'bc'
    
    Returns:
        a: The scattering length in Bohr radii.
    """

    if species == "ab":
        abg = -1582
        delta = -262.3
        B0 = 832.18
    elif species == "ac":
        abg = -1770
        delta = -166.6
        B0 = 689.68
    elif species == "bc":
        abg = -1490
        delta = -222.3
        B0 = 809.76
    else:
        return "Error: Invalid species argument, must be ab, ac, or bc"
    
    return abg * (1 - delta / (B - B0))

Function for calculating the scattering length of one Li atom and one Cs atom given the magnetic field strength for various species.

The possible species are "a" or "b" which represent the |Li:ground>-|Cs:Ground> and |Li:Excited>-|Cs:Ground> states respectively

The constants are pulled from tables 3.2 and 3.3 of:

    FESHBACH AND EFIMOV RESONANCES IN A $^{6}\mathrm{Li}-^{133}\mathrm{Cs}$ ATOMIC MIXTURE by Jacob Johansen

In [125]:
def aLiCs(B, species):
    """
    aLiCs - returns the Li-Cs scattering length at a given magnetic field B, for Li atoms in the state given by species. 

    Parameters:
        B: The magnetic field in Gauss.
        species: The state of the lithium atom, "a" for ground, and "b" for excited
    
    Returns:
        a: The scattering length in Bohr radii.
    """
    
    if species == 'a':
        return -29.4 * (1 - (-58.21)/(B - 842.829) - (-4.55)/(B - 892.648))
    
    elif species == 'b':
        return -29.6 * (1 - (-0.37)/(B - 816.113) - (-57.45)/(B - 888.577) - (-4.22)/(B - 943.033))
    
    else:
        return "Error: Invalid species argument, must be ab, ac, or bc"

In [6]:
def aLiCs_ci(abf, fieldErr, invFlag:bool):
    """
    Calculate the confidence interval on the LiCs scattering length abf given the field error fielderr.
    If invflag is true, then abf and sabf will be interpreted as 1/abf and 1/sabf.
    """

    pass

Function for calculating the scattering length of different LiCs moleculat species by spline interpolation.

The data is taken from LiCs_a_vs_B.txt.

Possible options are "a", "b", or "c" which represent $LiCs$, $Li_2Cs$, and $Li_3Cs$ respectively.



In [21]:
def aLiCs_molscat(B, species):
    """
    aLiCs_molscat - returns scattering length a from molscat calculations stored in file LiCs_a_cs_B.txt

    Parameters:
        B: The magnetic field in gauss
        species: The LiCs molecular species, "a" for LiCs, "b" for Li2Cs, or "c" for Li3Cs

    Returns:
        a: The scattering length of the chosen molular species in Bohr radii
    """

    BVals = LiCs_aVsBData[:, 0]
    
    if species == "a":
        #get the LiCs scattering values
        aVals = LiCs_aVsBData[:, 1]
    
    elif species == "b":
        #get the Li2Cs scattering values
        aVals = LiCs_aVsBData[:, 2]
    
    elif species == "c":
        #get the Li3Cs scattering values
        aVals = LiCs_aVsBData[:, 3]
    
    else:
        return "Error: Invalid species, must be a, b, or c"

    spline = sp.interpolate.CubicSpline(BVals, aVals, extrapolate=False)

    return spline(B)